In [0]:
%reload_ext autotime
FORECAST_TERM_CODE = 202508
LAG = 1
exec(open('./amp.py').read())
from sklearn.metrics import RocCurveDisplay
from sklearn.calibration import CalibrationDisplay

aggregates = union(
    'styp_desc','camp_desc',
    # 'coll_desc','dept_desc','majr_desc','hs_qrtl','tsi_math','tsi_reading','tsi_writing','resd_desc','oriented','waiver','lgcy',
    # 'gender',races,'international',
)

features = {'act_equiv':pd.NA,'age':pd.NA,'camp_desc':'stephenville','drivetime':pd.NA,'eager':pd.NA,'fafsa': False,'finaid': False,'gap_score':0,'gender':pd.NA,'hs_qrtl':pd.NA,'international':False,'lgcy':False,'oriented':False,**{r: False for r in races},'schlship':False,'ssb':False,'styp_desc':'new first time','_tot_sch':0,'tsi_math':False,'tsi_reading':False,'tsi_writing':False,'verified':False,'waiver':False}

crse_codes = union('_headcnt','_proba',
    'agec2317','ansc1119','ansc1319','anth2302','anth2351','arts1301','arts1303','arts1304','arts3331','biol1305','biol1406','biol1407','biol2401','biol2402','busi1301','busi1307','chem1111','chem1112','chem1302','chem1311','chem1312','chem1407','chem1409','cnst1301','comm1311','comm1315','comm2302','crij1301','dram1310','dram2361','easc2310','econ1301','econ2301','educ1301','engl1301','engl1302','engl2307','engl2320','engl2321','engl2326','engl2340','engl2350','engl2360','engl2362','engl2364','engl2366','engl2368','engr1211','engr2303','envs1302','fina1360','geog1303','geog1320','geog1451','geog2301','geol1403','geol1404','geol1407','geol1408','govt2305','govt2306','hist1301','hist1302','hist2321','hist2322','huma1315','kine1301','kine1338','kine2315','math1314','math1316','math1324','math1325','math1332','math1342','math1352','math2412','math2413','musi1303','musi1310','musi1311','musi2350','musi3325','phil1301','phil1304','phil2303','phil3301','phys1302','phys1401','phys1402','phys1403','phys1411','phys2425','phys2426','psyc2301','psyc3303','psyc3307','soci1301','soci1306','soci2303','univ0010','univ0200','univ0204','univ0301','univ0314','univ0324','univ0332','univ0342',)

REFRESHED = []
REFRESH = union(
    # 'flags',
    # 'admissions',
    # 'registrations',
    # 'students',
    'imputed',
    'learners',
    'predictions',
    'enrollments',
    'enrollmentsall',
    'studentsall',
    'predictionsall',
    'proba',
    'forecasts',
)

kwargs = dict(features=features, aggregates=aggregates, crse_codes=crse_codes, subpops='styp_desc', start=2022,
    # date='2025-10-30',
    flaml=dict(
        time_budget=60,
    ),)
# kwargs |= dict(crse_codes=['_headcnt','_proba'], aggregates='camp_desc'}  #quick for testing
if LAG > 0: #retention
    kwargs |= dict(crse_codes='_proba', aggregates=[])  #quick for testing
self = AMP(**kwargs)
# self.get_flagsday() # GA's should not have permissions to run this because it sees pii
self.get_proba()
# self.get_reports()   # may hit memory limits running get_flagsyear - just restart kernel and try again


def run_viz():
    P = self.get_proba().query(f'cohort_term != @self.term_desc & _tot_sch > 0')
    viz = {
        'ROC': lambda df: RocCurveDisplay.from_predictions(df['actual'], df['prediction'], ax=ax, name=df.name),
        'Calibration': lambda df: CalibrationDisplay.from_predictions(df['actual'], df['prediction'], ax=ax, name=df.name, n_bins=10)#, strategy='quantile'),
    }
    with no_warn():
        sns.displot(data=P, x='prediction', hue='actual', kind='kde', row='cohort_term', col='model_term', common_norm=False)
        for nm, fcn in viz.items():
            for cohort_term, B in P.groupmy('cohort_term', sort=True):
                fig, ax = plt.subplots()
                B.groupmy('model_term', sort=True).apply(fcn)
                ax.plot([0, 1], [0, 1], 'k--')
                ax.set_title(f'{nm}: {cohort_term} cohort {LAG} term retention')
                plt.show()

In [0]:
#train all
run_viz()

In [0]:
#train tot_sch > 0
run_viz()

In [0]:
P = self.get_proba().query(f'cohort_term != @self.term_desc & _tot_sch > 0')
viz = {
    'ROC': lambda df: RocCurveDisplay.from_predictions(df['actual'], df['prediction'], ax=ax, name=df.name),
    # 'Calibration': lambda df: CalibrationDisplay.from_predictions(df['actual'], df['prediction'], ax=ax, name=df.name, n_bins=10)#, strategy='quantile'),
}
with no_warn():
    sns.displot(data=P, x='prediction', hue='actual', kind='kde', row='cohort_term', col='model_term', common_norm=False)
    for nm, fcn in viz.items():
        for cohort_term, B in P.groupmy('cohort_term', sort=True):
            fig, ax = plt.subplots()
            B.groupmy('model_term', sort=True).apply(fcn)
            ax.plot([0, 1], [0, 1], 'k--')
            ax.set_title(f'{nm}: {cohort_term} cohort {LAG} term retention')
            plt.show()